In [33]:
import requests



In [34]:
path = "/healthz"
base_url = "http://localhost:8002" 
endpoint= f"{base_url}{path}"
endpoint
 



'http://localhost:8002/healthz'

In [35]:
response = requests.get(endpoint)
print(response.ok )
print(response.text)


True
{"status":"healthy"}


In [5]:
path = "/api/events"
base_url = "http://localhost:8002" 
endpoint= f"{base_url}{path}"
endpoint

response = requests.get(endpoint)
if response.ok:
    data = response.json()
    print(data)
    print(response.text)


else:
    print(response.text)
    print(response.status_code)

{'results': [{'id': 1, 'page': '', 'description': ''}, {'id': 2, 'page': '', 'description': ''}, {'id': 3, 'page': '', 'description': ''}], 'count': 3}
{"results":[{"id":1,"page":"","description":""},{"id":2,"page":"","description":""},{"id":3,"page":"","description":""}],"count":3}


In [105]:
path = "/api/events/2"
base_url = "http://localhost:8002" 
endpoint= f"{base_url}{path}"
endpoint


response = requests.get(endpoint)
if response.ok:
    data = response.json()
    print(response.headers )
    print(data, type(data))
    print(response.text)
    


else:
    print(response.status_code)
    print(response.text)
    print(endpoint)


{'date': 'Mon, 12 Jan 2026 20:27:49 GMT', 'server': 'uvicorn', 'content-length': '40', 'content-type': 'application/json'}
{'page': '/test', 'description': '', 'id': 2} <class 'dict'>
{"page":"/test","description":"","id":2}


In [59]:
import json


In [160]:
#POST create event 

data = {"page":"/test"}


path = "/api/events/"
base_url = "http://localhost:8002" 
create_endpoint= f"{base_url}{path}"
create_endpoint


response = requests.post(create_endpoint, json=data)

if response.ok:
    data = response.json()
    print(response.ok)
    print(response.headers )
    print(data, type(data))
    
    
else:
    print(response.status_code)
    print(response.ok)
    print(response.text)
    print(endpoint)




True
{'date': 'Tue, 13 Jan 2026 05:24:55 GMT', 'server': 'uvicorn', 'content-length': '41', 'content-type': 'application/json'}
{'page': '/test', 'description': '', 'id': 22} <class 'dict'>


In [164]:
#PUT update event
from pprint import  pprint

path = "/api/events"
base_url = "http://localhost:8002" 
detail_endpoint= f"{base_url}{path}"
endpoint


response = requests.get(detail_endpoint, json={"description":"updated via api"})

if response.ok:
    data = response.json()
    print(response.ok)
    print(response.headers )
    pprint(data)

    
    
else:
    print(response.status_code)
    print(response.ok)
    print(response.text)
    print(endpoint)



True
{'date': 'Tue, 13 Jan 2026 05:25:16 GMT', 'server': 'uvicorn', 'content-length': '730', 'content-type': 'application/json'}
{'count': 17,
 'result': [{'description': '', 'id': 1, 'page': '/test'},
            {'description': '', 'id': 4, 'page': '/test'},
            {'description': '', 'id': 5, 'page': '/test'},
            {'description': '', 'id': 6, 'page': '/test'},
            {'description': '', 'id': 7, 'page': '/test'},
            {'description': '', 'id': 8, 'page': '/test'},
            {'description': '', 'id': 9, 'page': '/test'},
            {'description': '', 'id': 10, 'page': '/test'},
            {'description': '', 'id': 11, 'page': '/test'},
            {'description': '', 'id': 12, 'page': '/test'},
            {'description': '', 'id': 13, 'page': '/test'},
            {'description': '', 'id': 14, 'page': '/test'},
            {'description': '', 'id': 15, 'page': '/test'},
            {'description': '', 'id': 16, 'page': '/test'},
            {'descriptio

In [159]:
#PUT update event
from pprint import  pprint

path = "/api/events/21"
base_url = "http://localhost:8002" 
detail_endpoint= f"{base_url}{path}"
endpoint


response = requests.put(detail_endpoint, json={"description":"updated via api"})

if response.ok:
    data = response.json()
    print(response.ok)
    print(response.headers )
    pprint(data)

    
    
else:
    print(response.status_code)
    print(response.ok)
    print(response.text)
    print(endpoint)



404
False
{"detail":"Event not found found"}
http://localhost:8002/api/events/2


In [163]:

from pprint import  pprint

path = "/api/events/22"
base_url = "http://localhost:8002" 
detail_endpoint= f"{base_url}{path}"
detail_endpoint


response = requests.delete(detail_endpoint)

    
    


